Make decoy datasets 
* input comprised of two sentences with agreeing sentiments in training but disagreeing in test ("This movie is very good. I like it a lot" vs "This movie is very good. The director must have been brain-dead")
* Have random agreeing words in train but disagree in test
* Key idea: structured noise 


In [57]:
%matplotlib inline
import os
from os.path import join as oj
import sys, time

import csv
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
from os.path import join
import torch
import torch
import numpy as np
import seaborn as sns
from copy import deepcopy
from model import LSTMSentiment
import matplotlib.pyplot as plt
from os.path import isdir

from torchtext.data import TabularDataset
# check out how two models differ
import torch.optim as O
import torch.nn as nn
from torchtext import data
from torchtext import datasets
from tqdm import tqdm_notebook, tqdm 
import pickle
from nltk import Tree
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import numpy as np
from torchtext import data
from torchtext import datasets
from nltk import Tree
import os
import csv
from os.path import join as oj
def write_dataset(name, noise =1.0):
    data_path = "./.data/sst/trees"
    os.makedirs(data_path, exist_ok=True)
    with open(oj(data_path, name + ".txt")) as f:
        content = []
        content = f.read().splitlines()
    parsed_dataset = []
    for line in (content):
        t = Tree.fromstring(line)
        text = t.leaves()
        label = int(t.label()) 
        if label !=2: 
            label = int(label >2)
            stop_position = [int(len(child.leaves())) for child in t][0]
            segment_labels = np.asarray([float(child.label()) for child in t])
            diff = segment_labels[0] - segment_labels[1]
            if label ==0:
                segment_labels = 4-segment_labels 
      
            if segment_labels.sum() ==0  or diff <3:
                segment_labels = np.asarray([0.5, 0.5])#np.round(np.asarray([ratio, 1- ratio]), decimals =1)
            else:
#                 segment_labels  =np.round(segment_labels/segment_labels.sum(), decimals =1)
                segment_labels =   (segment_labels == segment_labels.max()).astype(np.float32)
            
            parsed_dataset.append((" ".join(text), label, segment_labels[0],segment_labels[1] , stop_position))
    file_path = "../data"
    file_name = name + '_segment_annotated_SST.csv'

    with open(os.path.join(file_path, file_name), 'w') as csv_file:
        writer = csv.writer(csv_file)
        for line in parsed_dataset:

            if np.random.uniform() <noise:
    
                writer.writerow(line)


write_dataset("train", noise = 1)

In [59]:

write_dataset("dev")
write_dataset("test")

In [60]:
file_path = "../data"

In [61]:
dataset_path = file_path

In [52]:
inputs = data.Field(lower=True)
answers = data.Field(sequential=False, unk_token=None)
segment1_label = data.Field(sequential=False, unk_token=None, dtype=torch.float16)
segment2_label = data.Field(sequential=False, unk_token=None, dtype=torch.float16)
stop_pos = data.Field(sequential=False, unk_token=None)

tv_datafields = [ ("text", inputs), 
                 ("label", answers), 
                 ("segment1_label", segment1_label), 
                 ("segment2_label", segment2_label), 
                 ("stop_pos", stop_pos)]
train, dev, test = TabularDataset.splits(
                           path=dataset_path,
                           train='train_segment_annotated_SST.csv', validation="dev_segment_annotated_SST.csv", test = "test_segment_annotated_SST.csv",
                           format='csv', 
                           skip_header=False, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                           fields=tv_datafields)


In [53]:
len(train)

6920

In [54]:
a = 0
is_neg =0
test_list = []
for i in range(len(train)):
    test_list.append(float(train[i].segment2_label))
 
    if float(train[i].segment2_label) !=0.5:
        print(" ".join(train[i].text), train[i].label)
        print(" ".join(train[i].text[:int(train[i].stop_pos)]), train[i].segment1_label)
        print()
        a+=1
        if int(train[i].label) ==0:
            is_neg +=1
            
 

print(a/len(train))
print(is_neg/a)

the gorgeously elaborate continuation of `` the lord of the rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director peter jackson 's expanded vision of j.r.r. tolkien 's middle-earth . 1
the gorgeously elaborate continuation of `` the lord of the rings '' trilogy 1.0

`` frailty '' starts out like a typical bible killer story , but it turns out to be significantly different -lrb- and better -rrb- than most films with this theme . 1
`` frailty '' starts out like a typical bible killer story , but it turns out to be significantly different -lrb- and better -rrb- than most films with this theme 1.0

a smart , steamy mix of road movie , coming-of-age story and political satire . 1
a smart , steamy mix of road movie , coming-of-age story and political satire 1.0

a terrific b movie -- in fact , the best in recent memory . 1
a terrific b movie -- in fact , the best in recent memory 1.0

winds up being both revelatory and narcissistic , achieving som

although i did n't hate this one , it 's not very good either . 0
although i did n't hate this one 0.0

another best of the year selection . 1
another best of the year selection 1.0

a compelling story of musical passion against governmental odds . 1
a compelling story of musical passion against governmental odds 1.0

a forceful drama of an alienated executive who re-invents himself . 1
a forceful drama of an alienated executive who re-invents himself 1.0

the best film of the year 2002 . 1
the best film of the year 2002 1.0

an enthralling , entertaining feature . 1
an enthralling , entertaining feature 1.0

a fresh , entertaining comedy that looks at relationships minus traditional gender roles . 1
a fresh , entertaining comedy that 1.0

a pretty funny movie , with most of the humor coming , as before , from the incongruous but chemically perfect teaming of crystal and de niro . 1
a pretty funny movie , with most of the humor coming , as before , from the incongruous but chemically p

In [40]:
0.007803468208092485 *(len(train))

54.0

In [ ]:
set(test_list)

In [ ]:
name = "train"

In [ ]:

data_path = "./.data/sst/trees"
os.makedirs(data_path, exist_ok=True)
with open(oj(data_path, name + ".txt")) as f:
    content = []
    content = f.read().splitlines()
parsed_dataset = []
for line in (content[:11]):
    t = Tree.fromstring(line)
    text = t.leaves()
    label = int(t.label()) 
    if label !=2: 
        label = int(label >2)
        stop_position = [int(len(child.leaves())) for child in t][0]


        segment_labels = np.asarray([float(child.label()) for child in t])
        diff = segment_labels[0] - segment_labels[1]

        if label ==0:
            segment_labels = 4-segment_labels 
        ratio = stop_position/len(text)
        segment_labels[0] =ratio * segment_labels[0]
        segment_labels[1]=segment_labels[1] *(1-ratio)

        #segment_labels = (segment_labels >2).astype(np.float32)

        if segment_labels.sum() ==0  or diff <2:
            segment_labels = np.asarray([0.5, 0.5])#np.round(np.asarray([ratio, 1- ratio]), decimals =1)
        else:
            segment_labels  =np.round(segment_labels/segment_labels.sum(), decimals =1)


        parsed_dataset.append((" ".join(text), label, segment_labels[0],segment_labels[1] , stop_position))


In [ ]:
t.leaves()

In [ ]:
t.label()

In [ ]:
t.pprint()

In [ ]:
for child in t:
    print(child.label())
    print(child.leaves())